# <center>Lab Sheet 8</center>

# <center>Self-Organizing Maps (SOM)<center>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_digits
from sklearn.preprocessing import MinMaxScaler
import torch
from torchvision import datasets, transforms

np.random.seed(42)

**Q1. Implement a 1D SOM for color clustering using RGB vectors.**

In [2]:
class SOM1D:
    def __init__(self, num_neurons=8, dim=3, lr=0.5, sigma=1.0, epochs=100):
        self.num_neurons = num_neurons
        self.dim = dim
        self.lr = lr
        self.sigma = sigma
        self.epochs = epochs
        self.weights = np.random.rand(num_neurons, dim)

    def _neighborhood(self, bmu_idx, t):
        d = np.abs(np.arange(self.num_neurons) - bmu_idx)
        return np.exp(-(d**2) / (2 * (self.sigma * (1 - t/self.epochs))**2))

    def fit(self, X):
        for t in range(self.epochs):
            for x in X:
                dists = np.linalg.norm(self.weights - x, axis=1)
                bmu_idx = np.argmin(dists)
                h = self._neighborhood(bmu_idx, t)
                self.weights += self.lr * h[:, np.newaxis] * (x - self.weights)

colors = np.array([
    [1,0,0],[0,1,0],[0,0,1],
    [1,1,0],[1,0,1],[0,1,1],[0.5,0.5,0.5]
])
som1d = SOM1D(num_neurons=8, epochs=200)
som1d.fit(colors)

plt.figure(figsize=(6,2))
plt.title("Q1: 1D SOM for Color Clustering")
for i,w in enumerate(som1d.weights):
    plt.scatter(i,0,c=w.reshape(1,-1),s=500)
plt.axis("off")
plt.savefig("lab8_q1_som1d_colors.png",dpi=150,bbox_inches="tight"); plt.close()

**Q2. Train a 2D SOM to cluster handwritten digits (MNIST subset).**

In [3]:
class SOM2D:
    def __init__(self, m=10, n=10, dim=64, lr=0.5, sigma=None, epochs=100):
        self.m, self.n, self.dim = m, n, dim
        self.lr = lr
        self.epochs = epochs
        self.sigma = sigma if sigma else max(m,n)/2
        self.weights = np.random.rand(m*n, dim)

    def _grid_dist(self, bmu_idx):
        bx, by = divmod(bmu_idx, self.n)
        gx, gy = np.meshgrid(np.arange(self.m), np.arange(self.n), indexing="ij")
        return (gx-bx)**2 + (gy-by)**2

    def fit(self,X):
        for t in range(self.epochs):
            for x in X:
                dists = np.linalg.norm(self.weights - x, axis=1)
                bmu = np.argmin(dists)
                gdist = self._grid_dist(bmu)
                h = np.exp(-gdist / (2*(self.sigma*(1-t/self.epochs))**2))
                h = h.reshape(-1,1)
                self.weights += self.lr * h * (x - self.weights)

# Use sklearn digits (8x8 subset of MNIST for speed)
digits = load_digits()
X_digits = MinMaxScaler().fit_transform(digits.data)
y_digits = digits.target

som2d = SOM2D(m=10,n=10,dim=64,epochs=50)
som2d.fit(X_digits)       

**Q3. Visualize SOM feature map and U-matrix distance using Matplotlib.**

In [4]:
def u_matrix(som, m, n):
    W = som.weights.reshape(m,n,-1)
    U = np.zeros((m,n))
    for i in range(m):
        for j in range(n):
            d = 0; cnt = 0
            for dx,dy in [(0,1),(1,0),(-1,0),(0,-1)]:
                if 0<=i+dx<m and 0<=j+dy<n:
                    d += np.linalg.norm(W[i,j]-W[i+dx,j+dy])
                    cnt += 1
            U[i,j] = d/cnt
    return U

U = u_matrix(som2d,10,10)
plt.imshow(U,cmap="bone")
plt.title("Q3: SOM U-Matrix")
plt.colorbar()
plt.savefig("lab8_q3_umatrix.png",dpi=150,bbox_inches="tight"); plt.close()

**Q4. Compare SOM vs KMeans for unsupervised image segmentation.**

In [5]:
from skimage import data
from skimage.transform import resize

img = resize(data.astronaut(),(64,64))
X_img = img.reshape(-1,3)

# SOM clustering
som_img = SOM1D(num_neurons=8,dim=3,epochs=100)
som_img.fit(X_img)
labels_som = np.argmin(((X_img[:,None,:]-som_img.weights[None,:,:])**2).sum(-1),axis=1)
seg_som = som_img.weights[labels_som].reshape(img.shape)

# KMeans clustering
km = KMeans(n_clusters=8, random_state=42).fit(X_img)
seg_km = km.cluster_centers_[km.labels_].reshape(img.shape)

fig,ax=plt.subplots(1,3,figsize=(12,4))
ax[0].imshow(img); ax[0].set_title("Original")
ax[1].imshow(seg_som); ax[1].set_title("Q4: SOM Segmentation")
ax[2].imshow(seg_km); ax[2].set_title("KMeans Segmentation")
for a in ax: a.axis("off")
plt.savefig("lab8_q4_som_vs_kmeans.png",dpi=150,bbox_inches="tight"); plt.close()

/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


**Q5. Implement Learning Vector Quantization (LVQ) with labeled data.**

In [6]:
class LVQ:
    def __init__(self, prototypes_per_class=1, lr=0.1, epochs=10):
        self.prototypes_per_class = prototypes_per_class
        self.lr = lr
        self.epochs = epochs

    def fit(self, X, y):
        classes = np.unique(y)
        self.prototypes = []
        self.proto_labels = []
        for c in classes:
            idx = np.where(y==c)[0][:self.prototypes_per_class]
            self.prototypes.append(X[idx])
            self.proto_labels.extend([c]*len(idx))
        self.prototypes = np.vstack(self.prototypes)

        for _ in range(self.epochs):
            for i,x in enumerate(X):
                dists = np.linalg.norm(self.prototypes - x, axis=1)
                j = np.argmin(dists)
                if self.proto_labels[j] == y[i]:
                    self.prototypes[j] += self.lr * (x - self.prototypes[j])
                else:
                    self.prototypes[j] -= self.lr * (x - self.prototypes[j])

    def predict(self,X):
        preds=[]
        for x in X:
            dists=np.linalg.norm(self.prototypes-x,axis=1)
            j=np.argmin(dists)
            preds.append(self.proto_labels[j])
        return np.array(preds)

lvq = LVQ(prototypes_per_class=2,epochs=20)
lvq.fit(X_digits,y_digits)
preds = lvq.predict(X_digits)
print("Q5: LVQ training accuracy:", accuracy_score(y_digits,preds))

Q5: LVQ training accuracy: 0.9120756816917084
